In [3]:
!pip install datasets
!pip install accelerate
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
import os
import glob
import numpy as np
import pandas as pd
from PIL import Image

import seaborn as sns
import matplotlib.image as img
import matplotlib.pyplot as plt

In [6]:
# from datasets import load_dataset
# from datasets.features import ClassLabel


# ds_videos = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/ml2/ML_project_2_course/datasets/dataset_shoplifting/frames")




In [7]:
# ds = ds_videos["train"].train_test_split(test_size=0.1)

In [8]:
# ds

In [9]:
# np.sum(ds['test']['labels'])

In [10]:

# ds_train1 = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/ml2/ML_project_2_course/datasets/Augmentation/horizontal/horizontal_train")
# ds_train2 = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/ml2/ML_project_2_course/datasets/first_dataset/train")

In [11]:
# ds_test1 = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/ml2/ML_project_2_course/datasets/Augmentation/horizontal/horizontal_test")

In [12]:
# ds_test2 = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/ml2/ML_project_2_course/datasets/first_dataset/test")

In [13]:
# ds_valid2 = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/ml2/ML_project_2_course/datasets/first_dataset/train")

In [14]:
# from datasets import DatasetDict
# ds = DatasetDict({
#     'train': ds_train['train'],
#     'test': ds_test['train'],
#})

In [15]:
# ds

In [16]:
# ds = ds.class_encode_column("labels")

In [17]:
# ex = ds['train'][400]
# ex

In [18]:
# ds['train'].num_rows

In [19]:
# image = ex['image']
# image

In [20]:
# labels = ds['train'].features['labels']
# labels

In [21]:
# labels.int2str(ex['labels'])

In [22]:
# def get_augmented_dict(ds):
#   augmented_pictures = []
#   for i in range(ds.num_rows):

In [23]:
# ex_cropped_images = recieve_box(ex['image'])

# ex_cropped_images

In [24]:
# ex_cropped_images[0]

In [25]:
# from tqdm.auto import tqdm

In [26]:
# ds_split_name = 'train'
# dict_train_detected = {'image' : [], 'labels': []}
# for i in tqdm(range(ds[ds_split_name].num_rows)):
#   img = recieve_box(ds[ds_split_name][i]['image'])
#   label = ds[ds_split_name][i]['labels']
#   if(len(img) > 0):
#     dict_train_detected['image'].append(img[0])
#     dict_train_detected['labels'].append(label)
#   # for box_img in img:
#   #   ds_train_detected[i] = {'image': box_img,'labels': label}


In [27]:
# ds_cur = ds_train1
# dict_train1 = {'image' : [], 'labels': []}
# for i in tqdm(range(ds_cur['train'].num_rows)):
#   img = recieve_box(ds_cur['train'][i]['image'])
#   label = ds_cur['train'][i]['labels']
#   if(len(img) > 0):
#     dict_train1['image'].append(img[0])
#     dict_train1['labels'].append(label)

In [28]:
# ds_cur = ds_train2
# dict_train2 = {'image' : [], 'labels': []}
# for i in tqdm(range(ds_cur['train'].num_rows)):
#   img = recieve_box(ds_cur['train'][i]['image'])
#   label = ds_cur['train'][i]['labels']
#   if(len(img) > 0):
#     dict_train2['image'].append(img[0])
#     dict_train2['labels'].append(label)

In [29]:
# ds_cur = ds_valid2
# dict_valid2 = {'image' : [], 'labels': []}
# for i in tqdm(range(ds_cur['train'].num_rows)):
#   img = recieve_box(ds_cur['train'][i]['image'])
#   label = ds_cur['train'][i]['labels']
#   if(len(img) > 0):
#     dict_valid2['image'].append(img[0])
#     dict_valid2['labels'].append(label)

In [30]:
# ds_cur = ds_test1
# dict_test1 = {'image' : [], 'labels': []}
# for i in tqdm(range(ds_cur['train'].num_rows)):
#   img = recieve_box(ds_cur['train'][i]['image'])
#   label = ds_cur['train'][i]['labels']
#   if(len(img) > 0):
#     dict_test1['image'].append(img[0])
#     dict_test1['labels'].append(label)

In [31]:
# ds_cur = ds_test2
# dict_test2 = {'image' : [], 'labels': []}
# for i in tqdm(range(ds_cur['train'].num_rows)):
#   img = recieve_box(ds_cur['train'][i]['image'])
#   label = ds_cur['train'][i]['labels']
#   if(len(img) > 0):
#     dict_test2['image'].append(img[0])
#     dict_test2['labels'].append(label)

In [32]:
# ds_split_name = 'train'
# dict_train_detected = {'image' : [], 'labels': []}
# for i in tqdm(range(ds[ds_split_name].num_rows)):
#   img = recieve_box(ds[ds_split_name][i]['image'])
#   label = ds[ds_split_name][i]['labels']
#   if(len(img) > 0):
#     dict_train_detected['image'].append(img[0])
#     dict_train_detected['labels'].append(label)

In [33]:
# ds_split_name = 'test'
# dict_test_detected = {'image' : [], 'labels': []}
# for i in tqdm(range(ds[ds_split_name].num_rows)):
#   img = recieve_box(ds[ds_split_name][i]['image'])
#   label = ds[ds_split_name][i]['labels']
#   if(len(img) > 0):
#     dict_test_detected['image'].append(img[0])
#     dict_test_detected['labels'].append(label)

In [34]:
# dict_train_detected = {'image': dict_train1['image'] + dict_train2['image'] + dict_valid2['image'],
#                       'labels': dict_train1['labels'] + dict_train2['labels'] + dict_valid2['labels']
#                       }

In [35]:
# dict_train_detected = {'image': dict_train1['image'] + dict_train2['image'] + dict_valid2['image'],
#                       'labels': dict_train1['labels'] + dict_train2['labels'] + dict_valid2['labels']
#                       }

In [36]:
# dict_test_detected = {'image': dict_test1['image'] + dict_test2['image'],
#                       'labels': dict_test1['labels'] + dict_test2['labels']
#                       }

In [37]:
# ds_split_name = 'test'
# dict_test_detected = {'image' : [], 'labels': []}
# for i in tqdm(range(ds[ds_split_name].num_rows)):
#   img = recieve_box(ds[ds_split_name][i]['image'])
#   label = ds[ds_split_name][i]['labels']
#   if(len(img) > 0):
#     dict_test_detected['image'].append(img[0])
#     dict_test_detected['labels'].append(label)

In [38]:
# from datasets import Dataset

# ds_train_detected_new = Dataset.from_dict(dict_train_detected)

# ds_train_detected_new

In [39]:
# from datasets import Dataset

# ds_test_detected_new = Dataset.from_dict(dict_test_detected)

# ds_test_detected_new

In [40]:
# from datasets import DatasetDict
# ds_detected = DatasetDict({
#     'train': ds_train_detected_new,
#     'test': ds_test_detected_new,
#     })

# ds_detected = ds_detected.class_encode_column("labels")

In [41]:
import torch
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
from PIL import Image

def crop_image(image, box):
    additional_area = 1/10
    x_min, y_min, x_max, y_max = box
    width, height = image.size
    box_width = x_max - x_min
    box_height = y_max - y_min
    x_min_2 = max(0, x_min - box_width*additional_area)
    y_min_2 = max(0, y_min - box_height*additional_area)
    x_max_2 = min(width, x_max + box_width*additional_area)
    y_max_2 = min(height, y_max + box_height*additional_area)
    area = (x_min_2, y_min_2, x_max_2, y_max_2)
    cropped_img = image.crop(area)
    return cropped_img

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.set_device(0)
model_detect = YOLO('yolov8n.pt')
model_detect.classes = [0]

def recieve_box(image):
    results = model_detect(source = image, classes=0, show = False, imgsz=640, conf=0.2, iou=0.4, save = False, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    cropped_images = []
    for box in boxes:
        cropped_images.append(crop_image(image, box))
    return cropped_images

100%|██████████| 6.23M/6.23M [00:00<00:00, 119MB/s]


In [42]:
device

'cuda'

In [57]:
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from datasets.features import ClassLabel
from tqdm.auto import tqdm

def get_zoomed_dataset(data_dir):
  ds_demo = load_dataset("imagefolder", data_dir=data_dir)

  dict_detected = {'image' : [], 'labels': []}
  for i in tqdm(range(ds_demo['train'].num_rows)):
    img = recieve_box(ds_demo['train'][i]['image'])
    label = ds_demo['train'][i]['labels']
    if(len(img) > 0):
      dict_detected['image'].append(img[0])
      dict_detected['labels'].append(label)

  return Dataset.from_dict(dict_detected)


def get_dataset(need_split_train, data_train_dirs, data_test_dirs):
  train_datasets = []
  for data_dir in data_train_dirs:
    train_datasets.append(get_zoomed_dataset(data_dir))

  test_datasets = []
  for data_dir in data_test_dirs:
    test_datasets.append(get_zoomed_dataset(data_dir))

  ###
  for i in range(len(need_split_train)):
    if need_split_train[i]:
      train_datasets[i] = train_datasets[i].train_test_split(test_size=0.1)

      test_datasets.append(train_datasets[i]["test"])
      train_datasets[i] = train_datasets[i]["train"]
  ###
  ds_test_detected = concatenate_datasets(test_datasets)
  ds_train_detected = concatenate_datasets(train_datasets)

  return DatasetDict({
    'train': ds_train_detected,
    'test': ds_train_detected,
  })

In [58]:
need_split_train = [False, False, True]

data_train_dirs = ["/content/drive/MyDrive/ml2/ML_project_2_course/datasets/Augmentation/horizontal/horizontal_train",
                   "/content/drive/MyDrive/ml2/ML_project_2_course/datasets/first_dataset/train",
                   "/content/drive/MyDrive/ml2/ML_project_2_course/datasets/dataset_shoplifting/frames"]

data_test_dirs = ["/content/drive/MyDrive/ml2/ML_project_2_course/datasets/Augmentation/horizontal/horizontal_test",
                  "/content/drive/MyDrive/ml2/ML_project_2_course/datasets/first_dataset/test"]


ds_detected = get_dataset(need_split_train, data_train_dirs, data_test_dirs)



Resolving data files:   0%|          | 0/2186 [00:00<?, ?it/s]

  0%|          | 0/2185 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/753 [00:00<?, ?it/s]

  0%|          | 0/752 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/11981 [00:00<?, ?it/s]

  0%|          | 0/11980 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [68]:
ds_detected = ds_detected.class_encode_column("labels")

Stringifying the column:   0%|          | 0/12652 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/12652 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/12652 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/12652 [00:00<?, ? examples/s]

In [59]:
ds_detected

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 12652
    })
    test: Dataset({
        features: ['image', 'labels'],
        num_rows: 12652
    })
})

In [60]:
from transformers import ViTImageProcessor

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [61]:
def process_example(example):
    inputs = processor(example['image'], return_tensors='pt')
    inputs['labels'] = example['labels']
    return inputs

In [62]:
process_example(ds_detected['train'][0])

{'pixel_values': tensor([[[[-0.3647, -0.3882, -0.4039,  ..., -0.7255, -0.7725, -0.8118],
          [-0.3647, -0.3882, -0.4039,  ..., -0.7490, -0.7882, -0.8275],
          [-0.3647, -0.3882, -0.4039,  ..., -0.7725, -0.8039, -0.8196],
          ...,
          [-0.3725, -0.2471, -0.2314,  ..., -0.5216, -0.5529, -0.5294],
          [-0.0588, -0.0667, -0.4275,  ..., -0.4980, -0.5137, -0.5294],
          [-0.2706,  0.1529, -0.2314,  ..., -0.5373, -0.5216, -0.5529]],

         [[-0.4039, -0.4118, -0.4275,  ..., -0.7333, -0.7725, -0.8118],
          [-0.4039, -0.4118, -0.4275,  ..., -0.7569, -0.7882, -0.8275],
          [-0.4039, -0.4118, -0.4275,  ..., -0.7804, -0.8039, -0.8196],
          ...,
          [-0.5922, -0.4824, -0.4824,  ..., -0.4667, -0.4980, -0.4745],
          [-0.2784, -0.3098, -0.6941,  ..., -0.4431, -0.4588, -0.4745],
          [-0.4902, -0.0980, -0.4980,  ..., -0.4824, -0.4667, -0.4980]],

         [[-0.4510, -0.4667, -0.4824,  ..., -0.7490, -0.7882, -0.8275],
          [-0

In [63]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['labels']
    return inputs

In [64]:
prepared_ds = ds_detected.with_transform(transform)

In [65]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [66]:
import numpy as np

from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-66-37548c35eacb>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [67]:
from transformers import ViTForImageClassification

labels = ds_detected['train'].features['labels'].names

model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

AttributeError: 'Value' object has no attribute 'names'

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/ml2/ML_project_2_course/weights2",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  save_total_limit=1,
  logging_steps=10,
  learning_rate=2e-4,
  weight_decay=0.01,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

In [ ]:
training_args.device

In [ ]:
from datasets import load_metric

metric = load_metric('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=processor,
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
prepared_ds['test']

In [ ]:
ex = ds_detected['test'][400]
ex['image']

In [ ]:
inputs = processor(ex['image'], return_tensors='pt')

In [ ]:
# logits = model(**inputs).logits

# logits

In [ ]:
logits = model(inputs['pixel_values'].to(device)).logits

logits

In [ ]:
 logits.argmax(-1).item()

In [ ]:
# array = np.array(ds_train["train"]['labels'])
# array.sum()/array.shape[0]

In [ ]:
softmax = torch.nn.Softmax()

def image_to_prob(img):
    inputs = processor(img, return_tensors='pt')
    logits = model(inputs['pixel_values'].to(device)).logits
    return float(softmax(logits)[0][1])

In [ ]:
logits.cpu().data.numpy()

In [ ]:
def get_logits(img):
    inputs = processor(img, return_tensors='pt')
    return model(inputs['pixel_values'].to(device)).logits.cpu().data.numpy()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
logits_test = []
for i in tqdm(range(0, ds_detected['test'].num_rows)):
  logits_test.append(get_logits(ds_detected['test'][i]['image']))
  #logits_test.append(image_to_prob(ds_detected['test'][i]['image']))

In [ ]:
preds = np.array(logits_test).argmax(-1).flatten()

In [ ]:
probs = []
for i in tqdm(range(0, ds_detected['test'].num_rows)):
  probs.append(image_to_prob(ds_detected['test'][i]['image']))

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(ds_detected['test']['labels'], probs)

In [ ]:

roc_auc_score(ds_detected['test']['labels'], preds)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay

PrecisionRecallDisplay.from_predictions(ds_detected['test']['labels'], probs)
plt.show()

In [ ]:
def image_to_prob(img):
    inputs = processor(img, return_tensors='pt')
    logits = model(inputs['pixel_values'].to(device)).logits
    return float(softmax(logits)[0][1])

In [ ]:
def crop_image(image, box):
    additional_area = 1/10
    x_min, y_min, x_max, y_max = box
    width, height = image.size
    box_width = x_max - x_min
    box_height = y_max - y_min
    x_min_2 = max(0, x_min - box_width*additional_area)
    y_min_2 = max(0, y_min - box_height*additional_area)
    x_max_2 = min(width, x_max + box_width*additional_area)
    y_max_2 = min(height, y_max + box_height*additional_area)
    area = (x_min_2, y_min_2, x_max_2, y_max_2)
    cropped_img = image.crop(area)
    return cropped_img

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def recieve_box(image):
    results = model_detect(source = image, classes=0, show = False, imgsz=640, conf=0.2, iou=0.4, save = False, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    cropped_images = []
    for box in boxes:
        cropped_images.append(crop_image(image, box))
    return cropped_images

In [ ]:
from google.colab.patches import cv2_imshow

import cv2
from ultralytics import YOLO
import random
from PIL import Image

def draw_bounding_boxes_without_id(frame, results):
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    classes = results[0].boxes.cls.cpu().numpy().astype(int)

    for box, clss in zip(boxes, classes):
        # Generate a random color for each object based on its ID
        if clss != 0:
            random.seed(int(clss)+8)
            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3],), color, 2)
            cv2.putText(
                frame,
                f"{model.model.names[clss]}",
                (box[0], box[1]),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (50, 255, 50),
                2,
            )
    return frame


def process_video_with_tracking(model, input_video_path, show_video=True, save_video=False, output_video_path="output_video.mp4"):
    # Open the input video file
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        raise Exception("Error: Could not open video file.")

    # Get input video frame rate and dimensions
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the output video writer
    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model.track(frame, iou=0.4, conf=0.25, persist=True, imgsz=608, verbose=False, tracker="bytetrack.yaml", classes=0)

        if results[0].boxes.id != None: # this will ensure that id is not None -> exist tracks
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)

            for box, id in zip(boxes, ids):
                # Generate a random color for each object based on its ID
                random.seed(int(id))
                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                additional_area = 1/10
                x_min, y_min, x_max, y_max = box
                width, height = frame_width, frame_height
                box_width = x_max - x_min
                box_height = y_max - y_min
                x_min_2 = max(0, x_min - box_width*additional_area)
                y_min_2 = max(0, y_min - box_height*additional_area)
                x_max_2 = min(width, x_max + box_width*additional_area)
                y_max_2 = min(height, y_max + box_height*additional_area)
                area = (x_min_2, y_min_2, x_max_2, y_max_2)
                cropped_img = Image.fromarray(frame, 'RGB').crop(area)

                prob = image_to_prob(cropped_img)

                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3],), color, 2)
                cv2.putText(
                    frame,
                    f"{prob:.2f}",
                    (box[0], box[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.70,
                    (0, 255, 255),
                    2,
                )

        if save_video:
            out.write(frame)

        if show_video:
            frame = cv2.resize(frame, (0, 0), fx=0.75, fy=0.75)
            #cv2.imshow("frame", frame)
            cv2_imshow(frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Release the input video capture and output video writer
    cap.release()
    if save_video:
        out.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()
    return results

# Example usage:
model1 = YOLO('yolov8m.pt')
model1.fuse()
results = process_video_with_tracking(model1, "/content/drive/MyDrive/ml2/ML_project_2_course/datasets/videos/Shoplifting033_x264.mp4", show_video=False, save_video=True, output_video_path="/content/drive/MyDrive/ml2/ML_project_2_course/datasets/videos_tracked/output_video1.mp4")